<a href="https://colab.research.google.com/github/Sabina-0219/Sentiment-Analysis-System/blob/main/Imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import urllib.request
import os
import tarfile
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import re
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Flatten

Using TensorFlow backend.


In [ ]:
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
! tar zxf aclImdb_v1.tar.gz

--2022-06-27 10:00:15--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  64.9MB/s    in 1.2s    

2022-06-27 10:00:16 (64.9 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
def rm_tags(text):
  re_tag = re.compile(r'<[^>]+>')
  return re_tag.sub('', text)

In [ ]:
def read_files(filetype):
  path = "/content/aclImdb/"
  file_list = []

  positive_path = path + filetype + "/pos/"
  for f in os.listdir(positive_path):
    file_list += [positive_path+f]
  
  negative_path = path + filetype + "/neg/"
  for f in os.listdir(negative_path):
    file_list += [negative_path+f]
  
  print("read", filetype, "files:", len(file_list))

  all_labels = ([1] * 12500 + [0] * 12500)

  all_texts = []
  for fi in file_list:
    with open(fi, encoding = 'utf8') as file_input:
      all_texts += [rm_tags(" ".join(file_input.readlines()))]
  return all_labels, all_texts

In [ ]:
y_train, train_text = read_files("train")

read train files: 25000


In [ ]:
y_test, test_text = read_files("test")

read test files: 25000


In [ ]:
train_text[0]

"I'll keep it short and brief, the people who wrote the story lines for this show are genius, the actors are just perfect for the roles they play (CJ's character is legendary) and they have so much chemistry on screen which makes it what it is, a very successful comedy.When i saw first saw the new episodes which is probably going back just over 6/7 months, i wondered what had happened to Paul. I was gutted to find out that he had died when i browsed Google. He was so funny and played his character to perfection, an over-protective dad, who likes to keep his daughters out of the limelight and away from boys.The comedy, i think, has gone from strength to strength, even without Paul in it.Plus, i think most people would enjoy this watching it."

In [ ]:
y_train[0]

1

In [ ]:
train_text[20000]

'"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn\'t matter what one\'s political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn\'t true. I\'ve seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don\'t exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we\'re treated to the site of Vincent Gallo\'s throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the s

In [ ]:
y_train[20000]

0

In [ ]:
token = Tokenizer(num_words = 2000)
token.fit_on_texts(train_text)

In [ ]:
print(token.document_count)

25000


In [ ]:
print(token.word_index)

{'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'is': 6, 'in': 7, 'it': 8, 'i': 9, 'this': 10, 'that': 11, 'was': 12, 'as': 13, 'for': 14, 'with': 15, 'movie': 16, 'but': 17, 'film': 18, 'on': 19, 'not': 20, 'you': 21, 'are': 22, 'his': 23, 'have': 24, 'be': 25, 'he': 26, 'one': 27, 'all': 28, 'at': 29, 'by': 30, 'an': 31, 'they': 32, 'who': 33, 'so': 34, 'from': 35, 'like': 36, 'her': 37, 'or': 38, 'just': 39, 'about': 40, "it's": 41, 'out': 42, 'has': 43, 'if': 44, 'some': 45, 'there': 46, 'what': 47, 'good': 48, 'more': 49, 'when': 50, 'very': 51, 'up': 52, 'no': 53, 'time': 54, 'she': 55, 'even': 56, 'my': 57, 'would': 58, 'which': 59, 'only': 60, 'story': 61, 'really': 62, 'see': 63, 'their': 64, 'had': 65, 'can': 66, 'were': 67, 'me': 68, 'well': 69, 'than': 70, 'we': 71, 'much': 72, 'been': 73, 'get': 74, 'bad': 75, 'will': 76, 'also': 77, 'do': 78, 'into': 79, 'people': 80, 'other': 81, 'first': 82, 'because': 83, 'great': 84, 'how': 85, 'him': 86, 'most': 87, "don't": 88, 'made

In [ ]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

In [ ]:
print(train_text[0])

I'll keep it short and brief, the people who wrote the story lines for this show are genius, the actors are just perfect for the roles they play (CJ's character is legendary) and they have so much chemistry on screen which makes it what it is, a very successful comedy.When i saw first saw the new episodes which is probably going back just over 6/7 months, i wondered what had happened to Paul. I was gutted to find out that he had died when i browsed Google. He was so funny and played his character to perfection, an over-protective dad, who likes to keep his daughters out of the limelight and away from boys.The comedy, i think, has gone from strength to strength, even without Paul in it.Plus, i think most people would enjoy this watching it.


In [ ]:
print(x_train_seq[0])

[636, 397, 8, 342, 2, 1412, 1, 80, 33, 1037, 1, 61, 407, 14, 10, 119, 22, 1258, 1, 152, 22, 39, 399, 14, 1, 551, 32, 293, 105, 6, 2, 32, 24, 34, 72, 1170, 19, 264, 59, 162, 8, 47, 8, 6, 3, 51, 1106, 208, 50, 9, 215, 82, 215, 1, 157, 668, 59, 6, 238, 166, 141, 39, 116, 1081, 689, 1916, 9, 47, 65, 570, 5, 718, 9, 12, 5, 165, 42, 11, 26, 65, 1126, 50, 9, 26, 12, 34, 158, 2, 252, 23, 105, 5, 31, 116, 1242, 33, 1226, 5, 397, 23, 42, 4, 1, 2, 241, 35, 1008, 1, 208, 9, 100, 43, 820, 35, 5, 56, 205, 718, 7, 8, 930, 9, 100, 87, 80, 58, 353, 10, 145, 8]


In [ ]:
x_train = sequence.pad_sequences(x_train_seq, maxlen = 200)
x_test = sequence.pad_sequences(x_test_seq, maxlen = 200)

In [ ]:
print('length before processing = ', len(x_train_seq[5]))
print(x_train_seq[5])

length before processing =  121
[1127, 6, 3, 342, 444, 19, 1, 8, 1195, 1, 1338, 114, 342, 18, 29, 1, 18, 1407, 2, 77, 1195, 1, 29, 1772, 1728, 1407, 50, 113, 1350, 6, 30, 337, 55, 2, 37, 591, 19, 3, 1157, 1306, 5, 74, 8, 141, 7, 1, 4, 1, 81, 495, 32, 107, 772, 101, 2, 1729, 395, 1190, 1127, 6, 31, 36, 159, 870, 122, 106, 41, 19, 8, 6, 51, 200, 303, 2, 3, 84, 277, 4, 2, 1127, 6, 51, 46, 22, 53, 410, 2, 514, 2, 34, 8, 6, 49, 31, 14, 1467, 70, 472, 32, 199, 20, 387, 29, 28, 3, 84, 1306, 14, 144, 33, 74, 1, 576, 5, 63, 10, 1551, 990, 31, 211, 63]


In [ ]:
print('length after processing = ', len(x_train[5]))
print(x_train[5])

length after processing =  200
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0 1127    6    3  342  444
   19    1    8 1195    1 1338  114  342   18   29    1   18 1407    2
   77 1195    1   29 1772 1728 1407   50  113 1350    6   30  337   55
    2   37  591   19    3 1157 1306    5   74    8  141    7    1    4
    1   81  495   32  107  772  101    2 1729  395 1190 1127    6   31
   36  159  870  122  106   41   19    8    6   51  200  303    2    3
   84  277    4    2 1127    6   51   46   22   53  410    2  514    2
   34    8    6   49   31   14 1467   70  472   32  199   20  387   29
   28    3   84 1306   14  144   33   74    1 

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(output_dim = 32,
                    input_dim = 2000,
                    input_length = 200))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(units = 256, 
                activation = 'relu'))

In [ ]:
model.add(Dense(units = 1, 
                activation = 'sigmoid'))

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 200, 32)           64000     
_________________________________________________________________
flatten_4 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               1638656   
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 1,702,913
Trainable params: 1,702,913
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [ ]:
train_history = model.fit(x_train, y_train, batch_size = 100, epochs = 10, verbose = 0, validation_split = 0.2)

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)

25000/25000 [==============================] - 1s 48us/step


In [ ]:
scores[1]

0.8386800289154053

In [ ]:
predict = model.predict_classes(x_test)

In [ ]:
predict[:20]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1]], dtype=int32)

In [ ]:
predict_classes = predict.reshape(-1)
predict_classes[:20]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1],
      dtype=int32)

In [ ]:
print(y_test[:20])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
SentimentDict = {1: "Positive Reviews", 0:"Negative Reviews"}

In [ ]:
def display_test_Sentiment(i):
  print(test_text[i])
  print('Label: ', SentimentDict[y_test[i]],
        'Prediction: ', SentimentDict[predict_classes[i]])

In [ ]:
display_test_Sentiment(50)

They were alternative before there was alternative, The Residents are a band like no other, and I love them for it. This has all their classics, from 'Hello Skinny', 'Third Reich and Roll' to their homage to the great James Brown with a take on 'This is a Man's Man's Man's World'. But that is just the beginning. As a bonus it even has Renaldo & the Loafs hauntingly beautiful 'Songs for Swinging Larvae' and even features The Residents cover of it. Needless to say, I highly recommend the purchase of this DVD, I would also recommend buying their latest album 'Demons Dance Alone', it is fantastic.Uncle Willie Eyeball Buddy #502
Label:  Positive Reviews Prediction:  Positive Reviews


In [ ]:
display_test_Sentiment(20050)

After looking for this Bruceploitation for months, and then accidentally buying it cheap, it was disappointing. I heard about it on a DVD-R site, and it sounded crazy.But no, what I got was a pretty bad martial arts movie. The kung fu-ing wasn't too bad, but the rest of the movie was pretty awful, and made the movie seem really, really long, much longer then it's 85 minute runtime.On a positive note, the ape was funny for a couple of seconds (especially when I think they took the only close up of it's face from another movie), and the black guy who pops up half way through was funny because of the one line he said, which singlehandedly made all black people look like complete simpletons.But it wasn't enough, sadly.4/10
Label:  Negative Reviews Prediction:  Negative Reviews


In [ ]:
def predict_review(input_text):
  input_seq = token.texts_to_sequences([input_text])
  pad_input_seq = sequence.pad_sequences(input_seq, maxlen = 200)
  predict_result = model.predict_classes(pad_input_seq)
  print(SentimentDict[predict_result[0][0]])  #2D array

In [ ]:
predict_review("This is an impressive movie.")

Positive Reviews
